In [2]:
%load_ext gprof2dot_magic
from sklearn import datasets, linear_model, neighbors, svm, ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from base import SuperLearner, BMA, try_super_learners
import pandas as pd
import numpy as np
from scipy import stats
from pyearth import Earth
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # warnings from py-earth

v_folds = 5
ols = linear_model.LinearRegression()
elnet = linear_model.ElasticNetCV(l1_ratio=0.5, cv=v_folds, normalize=True)
ridge = linear_model.RidgeCV(cv=v_folds)
lars = linear_model.LarsCV(cv=v_folds, normalize=True)
lasso = linear_model.LassoCV(cv=v_folds, normalize=True)
nn = neighbors.KNeighborsRegressor(weights='distance')
svm1 = svm.SVR(kernel='linear', C=10, gamma='auto')
svm2 = svm.SVR(kernel='poly', C=10, gamma='auto')
rf = ensemble.RandomForestRegressor(n_estimators=100,min_samples_split=5)
gbm = ensemble.GradientBoostingRegressor()
# earth is sort of like D/S/A?
earth=Earth(max_terms=50,max_degree=3,use_fast=True,verbose=0) # get this from https://github.com/scikit-learn-contrib/py-earth
rtree=DecisionTreeRegressor(max_depth=3,min_samples_split=5)

cands=[ols,lars,earth,rf]
metas=[ols,lasso,ridge,earth,rf,rtree,gbm]
def helper(X1,y1,X2,y2,cands_bma=cands,cands_sl=cands,metas=metas):
    display(try_super_learners(cands_sl,metas,X1,y1,X2,y2))
#     sl=SuperLearner(cand_learners=cands,V=10,meta_learner=ols).fit(X1,y1)
#     for meta in metas:
#         sl.meta_learner_=meta.fit(sl.Z_train_cv_,y1)
#         df=sl.debug(X1,y1,X2,y2,skip_fit=True)
        
    display(BMA(cand_learners=cands_bma).debug(X1,y1,X2,y2))

The gprof2dot_magic module is not an IPython extension.


c:\program files\python36\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
c:\program files\python36\lib\site-packages\numpy\.libs\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll
c:\program files\python36\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [17]:
# first simulation study
def sim1(n):
    w=np.random.binomial(1,.4,size=(10,n))
    eps=np.random.normal(0,1,size=n)
    y=2*w[0]*w[9]+4*w[1]*w[6]+3*w[3]*w[4]-\
    5*w[5]*w[9]+3*w[7]*w[8]+w[0]*w[1]*w[3]-\
    2*w[6]*(1-w[5])*w[1]*w[8]-4*(1-w[9])*w[0]*(1-w[3])+eps
    return np.transpose(w),y

train1,test1=sim1(500),sim1(10000)
helper(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,6.254280,6.357725,6.427383
1,LarsCV,6.254786,6.356350,6.426881
2,Earth,1.155605,1.140024,1.178035
3,RandomForestRegressor,0.764694,1.471861,1.295168
4,Meta (LinearRegression),1.031271,1.124376,1.136143
5,Meta (LassoCV),1.035281,1.125130,1.137943
6,Meta (RidgeCV),1.027402,1.124398,1.135209
7,Meta (Earth),1.038735,1.115239,1.140714
8,Meta (RandomForestRegressor),1.107668,0.480576,1.264193
9,Meta (DecisionTreeRegressor),1.281951,1.233173,1.315497


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,6.254280,6.427383,0.479776
1,LarsCV,6.254786,6.426881,0.520224
2,Earth,1.155605,1.178035,0.000000
3,RandomForestRegressor,0.765175,1.295900,0.000000
4,BMA,6.254417,6.427003,NaN


In [136]:
# Second simulation (low noise linear)
def sim2(n, noise_ratio=0.1):
    w = np.zeros(12)
    w[0:6] = 0.9
    w[6:8] = 0.4
    w[8:10] = 0.2
    x12 = np.random.poisson(1 ,size=(2,n))
    x36 = np.random.uniform(0,1, size=(4,n))
    x78 = x12*x36[0:2]
    x910 = x36[0:2]*x36[1:3]
    x1112 = np.random.binomial(2, 0.5, size=(2,n))
    y_mat = pd.DataFrame(np.transpose(np.concatenate([x12, x36, x78, x910, x1112], axis=0)))
    X = y_mat[y_mat.columns[[0,1,2,3,4,5,10,11]]]
    Ey = y_mat.multiply(w, axis=1).sum(axis=1)
    var_y = np.var(Ey)
    eps = np.random.normal(0,noise_ratio*var_y,size=n)
    y = Ey + eps
    return X, y
        
train2,test2=sim2(1000,0.1),sim2(10000,0.1)
helper(*train2,*test2)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,0.122939,0.126714,0.111752
1,LarsCV,0.123065,0.126651,0.111598
2,Earth,0.090426,0.095979,0.088007
3,RandomForestRegressor,0.051434,0.319362,0.240013
4,Meta (LinearRegression),0.092269,0.095804,0.087676
5,Meta (LassoCV),0.090528,0.095868,0.087592
6,Meta (RidgeCV),0.091784,0.095843,0.087547
7,Meta (Earth),0.089617,0.095325,0.088724
8,Meta (RandomForestRegressor),0.071876,0.024643,0.109459
9,Meta (DecisionTreeRegressor),0.168580,0.175111,0.197187


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,0.122939,0.111752,3.745924e-01
1,LarsCV,0.123065,0.111598,6.254076e-01
2,Earth,0.090426,0.088007,7.503363e-68
3,RandomForestRegressor,0.051721,0.243867,3.150094e-189
4,BMA,0.122989,0.111624,NaN


In [95]:
# third simulation (linear high noise)
train3,test3=sim2(1000,0.35),sim2(10000,0.35)
helper(*train3,*test3)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,1.225370,1.251958,1.131918
1,LarsCV,1.225370,1.256823,1.131918
2,Earth,1.146652,1.310280,1.174984
3,RandomForestRegressor,0.316527,1.581722,1.332786
4,Meta (LinearRegression),1.182591,1.240126,1.121510
5,Meta (LassoCV),1.186786,1.241117,1.125024
6,Meta (RidgeCV),1.181947,1.241823,1.127831
7,Meta (Earth),1.225412,1.251915,1.130810
8,Meta (RandomForestRegressor),1.413252,0.286638,1.316522
9,Meta (DecisionTreeRegressor),1.314001,1.259405,1.273422


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,1.225370,1.131918,5.000000e-01
1,LarsCV,1.225370,1.131918,5.000000e-01
2,Earth,1.146652,1.174984,1.909966e-15
3,RandomForestRegressor,0.312295,1.343668,6.105197e-298
4,BMA,1.225370,1.131918,NaN


In [159]:
# non-linear simulation (low noise)
def sim3(n, noise_ratio=0.2):
    # not finished
    x14 = np.random.binomial(1,.4,size=(4,n))
    x48 = np.random.binomial(8, 0.2, size=(4,n))
    x912 = np.random.normal(2, 2, size=(4,n))

    X = np.transpose(pd.DataFrame(np.concatenate([x14,x48,x912])))
    Ey = 0.4*(x48[1]> 3)*(x48[2] < 3) + x14[1]*x14[0]*(4-x48[2])\
        - x48[1]*0.1*x912[0] + 0.5*x912[3]*((x912[2]>0)*(x912[1]>6)) + x48[1]*(x14[1])\
        + 0.5*x912[1]*(x48[3]>2)*x48[3] + (1-x14[0])*(1+x48[2])

    var_y = np.var(Ey)
    eps = np.random.normal(0,noise_ratio*var_y,size=n)
    y = Ey + eps
    return X, y

train4,test4=sim3(1000,0.1),sim3(10000,0.1)
helper(*train4,*test4)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,4.957966,5.238206,4.661929
1,LarsCV,4.978674,5.211162,4.649952
2,Earth,1.270930,1.587444,1.378822
3,RandomForestRegressor,0.334369,1.740477,1.784876
4,Meta (LinearRegression),0.538267,1.173914,1.093917
5,Meta (LassoCV),0.540778,1.174792,1.092952
6,Meta (RidgeCV),0.539561,1.174401,1.093059
7,Meta (Earth),0.658566,1.073289,1.149493
8,Meta (RandomForestRegressor),0.686872,0.289114,1.212570
9,Meta (DecisionTreeRegressor),1.180555,1.389833,1.570967


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,4.957966,4.661929,1.106580e-01
1,LarsCV,4.978674,4.649952,8.893420e-01
2,Earth,1.270930,1.378822,2.837904e-297
3,RandomForestRegressor,0.338483,1.760536,0.000000e+00
4,BMA,4.974345,4.649270,NaN


In [158]:
# non-linear simulation (high noise)
train5,test5=sim3(1000,0.35),sim3(10000,0.35)
helper(*train5,*test5)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,12.008361,12.542287,12.359046
1,LarsCV,12.077638,12.459610,12.406184
2,Earth,9.163922,10.640488,9.150718
3,RandomForestRegressor,2.115173,10.767616,10.179206
4,Meta (LinearRegression),4.797726,9.672476,8.969340
5,Meta (LassoCV),4.828915,9.682608,8.939522
6,Meta (RidgeCV),4.793133,9.673175,8.959912
7,Meta (Earth),5.112792,9.508858,8.934766
8,Meta (RandomForestRegressor),6.715342,2.293275,9.941115
9,Meta (DecisionTreeRegressor),6.426708,9.426961,9.865818


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,12.008361,12.359046,5.334062e-02
1,LarsCV,12.077638,12.406184,9.466594e-01
2,Earth,9.163922,9.150718,1.061390e-60
3,RandomForestRegressor,2.107758,10.193866,0.000000e+00
4,BMA,12.070445,12.400462,NaN


In [6]:
diabetes=datasets.load_diabetes()

X_train, X_test, y_train, y_test = train_test_split(
    diabetes.data, diabetes.target, test_size=0.2)

helper(X_train,y_train,X_test,y_test)

/home/hurichardj/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,2924.180414,3089.363281,2709.342336
1,LarsCV,3013.342624,3152.375809,2633.225901
2,Earth,2432.672601,3377.575004,3371.050633
3,RandomForestRegressor,656.902279,3505.297794,2909.303762
4,Meta (LinearRegression),2530.162268,3054.640262,2707.847372
5,Meta (LassoCV),2538.036979,3054.696219,2710.880546
6,Meta (RidgeCV),2530.112087,3054.640263,2707.843733
7,Meta (Earth),4233.243797,2587.447943,3228.879625
8,Meta (RandomForestRegressor),2827.377559,676.843566,3305.638772
9,Meta (DecisionTreeRegressor),3408.713926,2632.611410,2999.237021


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,2924.180414,2709.342336,4.960352e-03
1,LarsCV,3013.342624,2633.225901,9.950396e-01
2,Earth,2432.672601,3371.050633,3.886370e-17
3,RandomForestRegressor,661.078828,2741.870309,5.030033e-117
4,BMA,3012.460266,2633.286333,NaN


In [3]:
pr=pd.read_csv("datasets/CASP.csv")
feature_cols= pr.columns[pr.columns!='RMSD']
from sklearn.preprocessing import scale
pr.loc[:,feature_cols]=scale(pr.loc[:,feature_cols])

In [4]:
prtrain,prtest=train_test_split(pr.sample(1000))

In [5]:
helper(prtrain.loc[:,feature_cols],prtrain.RMSD,prtest.loc[:,feature_cols],prtest.RMSD)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,26.326647,28.782746,27.900947
1,LarsCV,27.000672,28.308667,28.373006
2,Earth,21.207329,36.210188,30.775924
3,RandomForestRegressor,4.323554,24.573168,24.343960
4,Meta (LinearRegression),6.427442,24.058655,23.580835
5,Meta (LassoCV),7.219783,24.137207,23.817534
6,Meta (RidgeCV),6.467205,24.058900,23.586689
7,Meta (Earth),12.016442,20.261697,26.809669
8,Meta (RandomForestRegressor),13.282699,5.696720,25.669582
9,Meta (DecisionTreeRegressor),14.050725,22.921058,28.382461


,Learner,Train MSE,Test MSE,Coefs
0,LinearRegression,26.326647,27.900947,9.500112e-295
1,LarsCV,27.000672,28.373006,7.254262e-299
2,Earth,21.207329,30.775924,1.562512e-259
3,RandomForestRegressor,4.329691,23.946793,1.000000e+00
4,BMA,4.329691,23.946793,NaN
